# **[:+:] Feature Selection + Nearest Neighbor [:+:]**

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as seab
from itertools import chain, combinations
#from scipy.spatial import minkowski_distance
from numba import jit, cuda
import numpy as np
from timeit import default_timer as timer

# - [x] Find "default Rate" : size(most common class)/size(dataset)
# - [x] Create Minkowski Distance function for two points
# - [x] Find nn of a single data point using only one feature
# - [] Get rid of spurrious percision
# - [x] For small data, find nn of every data point using one feature + max accuracy
# - [x] For small data, find nn of every data point using all features + max accuracy
# - [x] Complete Forward selection for small datas set
# - [] Complete Backward Elimination for small data set
# - [] Multithreading
# - [] GPU Multithreading
# - [x]] Leave-One-Out Cross Validation
# - [] Data Viz
# - [] Report
# - [] Find more brown m&ms
# - [] Clean up the code

### **[+] Reading the Data**

In [55]:
#Read data from the text files and set as a pandas data frame
lilData = pd.read_csv('CS170_Small_Data__19.txt', sep="  ", engine='python', header=None)
miniData = pd.read_csv('CS170_Smallified_Data__19.txt', sep="  ", engine='python', header=None)

#print all the data:
#print(lilData)


#----print just the 'label' column (and indeces):
print("The first 4 entries of the first column: data[colIdx][rowIdx]:")
print(lilData[0][0:5])


print("Matrix Dimensions:")
lil_rowXcol= lilData.shape
print(lil_rowXcol)


print("Data Head:")
print(lilData.head())

print("MetaData: ")
lilData.describe()



print('Occurrence counts of classes:')
# count occurrences of the class column
occur = lilData.groupby([0]).size()
# display occurrences of a particular column
display(occur)
lilDefaultRate= 405/500
print("Default Rate:")
print(lilDefaultRate)


print("\nConvert to numpy array to enable multithreading later:")
lilData= lilData.to_numpy()
print("column size after conversion: " + str(len(lilData[0])))
lilData= lilData.transpose()
print("column size after transposition: " + str(len(lilData[0])))
print("Dimensions after transposition: " + str(lilData.shape))



The first 4 entries of the first column: data[colIdx][rowIdx]:
0    2.0
1    1.0
2    2.0
3    2.0
4    2.0
Name: 0, dtype: float64
Matrix Dimensions:
(500, 7)
Data Head:
     0         1         2         3         4         5         6
0  2.0 -1.101866 -0.782026  0.552502  0.454685  1.132363  1.135458
1  1.0  0.928979 -0.169694  1.465293 -1.591929  0.144808  0.162709
2  2.0  1.123118 -1.384730 -0.903598  0.692522  0.669263 -0.142156
3  2.0  0.816617 -0.043628  1.026966  0.231013 -0.006551  2.316509
4  2.0 -1.159129 -1.341375  0.459997  0.631261 -1.479455  0.520158
MetaData: 
Occurrence counts of classes:


0
1.0     95
2.0    405
dtype: int64

Default Rate:
0.81

Convert to numpy array to enable multithreading later:
column size after conversion: 7
column size after transposition: 500
Dimensions after transposition: (7, 500)


### **[+] Minkowski Distance Calculation**

In [56]:
#My (from-scratch)Function takes in indices of two data points and exponent p, returns the minkowski distance between them based on the four dimensions and the exponent
# @jit(target_backend='cuda')
def minkowski(data,rowIdx1,rowIdx2,ftSet,p):
    # featSet1= []
    # featSet2= []
    sigma= 0
    for ft in ftSet:
        # featSet1.append(data[ft][rowIdx1])
        # featSet2.append(data[ft][rowIdx2])
        sigma+= ((((data[ft][rowIdx1]-data[ft][rowIdx2])**2))**(1/2))  **(p)
        distance= sigma**(1/p)
    return distance
    # return distance, featSet1, featSet2

# d= minkowski(lilData, rowIdx1=1, rowIdx2= 2, ftSet=(1,2,3,4,5,6), p=2)
# print(d)
# minkowski_distance(d[1],d[2],p=2)

## **[:+:] Forward Selection**

In [57]:
# @jit(target_backend='cuda')
def generateCombos(data):
    features= []
    ftRow= data.shape[0]
    for i in range(1,ftRow):
        features.append(i)
    print(features)
    return chain.from_iterable(combinations(features, r) for r in range(len(features) + 1))
    

# @jit(target_backend='cuda')
def generateDefaultRate(data):
    classCnt= [0,0]
    for pt in data[0]:
        if pt==1:
            classCnt[0]+= 1
        else:
            classCnt[1]+= 1
    defaultRate= max(classCnt)/(classCnt[0]+classCnt[1])
    return defaultRate


# takes a single combo of features, classifies each dp by its nearest neighbor, returns a list of classifications for every dp
# @jit(target_backend='cuda')
def classifier(data, ftSet):
    rows= data.shape[1]
    nearestNeighbors= []
    for dp in range(0,rows):
        nnIdx= 0
        nearest= 10000
        classification= 0
        for  neighborIdx in range(0,rows):
            if dp != neighborIdx:
                dist= minkowski(data, dp, neighborIdx, ftSet, p=2)
                if dist < nearest:
                    nnIdx= neighborIdx
                    nearest= dist
        classification= data[0][nnIdx]
        nearestNeighbors.append(classification)
    return nearestNeighbors


#  Returns the accuracy of the classifier tested with a combination of features (leave-one-out, k=n)
# @jit(target_backend='cuda')
def crossValidate(data, ftCombos):
    classes= classifier(data, ftSet= ftCombos)
    classesL= len(classes)
    correct= 0
    for j in range(0,classesL):
        if classes[j] == data[0][j]:
            correct+=1
    accuracy = correct/classesL
    return accuracy


# @jit(target_backend='cuda')
def forwardSelection(data, prevFtSet=[],finalAccuracy=0, finalSet=[]):
    ftSetLen= len(prevFtSet)
    ftRowLength= data.shape[0]
    if ftSetLen>= ftRowLength-1:
        bestFtSet= finalSet
        print("[:+:] ---  Forward Selection Results: The best set of features is " +str(bestFtSet)+ ", | Accuracy: %" +str(round(finalAccuracy*100,3))+ "  --- [:+:]")
    else:
        print("[::] -- Forward Selection Search Tree Level "+str(ftSetLen+1)+ " -- [::]" )
        if ftSetLen==0:
            print("        - Testing without features  | Default Rate: %" + str(round(generateDefaultRate(data)*100,3)))
        bestAcc= 0
        bestFtSubset= []
        for ft in range(1, ftRowLength):
            ftSet= []
            if ft not in prevFtSet:
                ftSet= prevFtSet + [ft]
                ftAccuracy= crossValidate(data,ftCombos= ftSet)
                print("        - Testing with features "+ str(ftSet) + " | Accuracy: %" + str(round(ftAccuracy*100,3)))
                if ftAccuracy>bestAcc: 
                    bestAcc= ftAccuracy
                    bestFtSubset= ftSet
        if bestAcc>finalAccuracy:
            finalSet= bestFtSubset
            finalAccuracy= bestAcc
        print("[+] -- On level "+str(ftSetLen+1)+ ", we add feature subset " + str(bestFtSubset)+" with an accuracy of %" + str(round(bestAcc*100,3))+ "\n")
        bestFtSet=  forwardSelection(data, bestFtSubset,finalAccuracy, finalSet)
    return bestFtSet





# start = timer()
print(forwardSelection(lilData))
# print("Time without GPU:", timer()-start)



[::] -- Forward Selection Search Tree Level 1 -- [::]
        - Testing without features  | Default Rate: %81.0
        - Testing with features [1] | Accuracy: %57.143
        - Testing with features [2] | Accuracy: %71.429
        - Testing with features [3] | Accuracy: %71.429
        - Testing with features [4] | Accuracy: %71.429
        - Testing with features [5] | Accuracy: %71.429
        - Testing with features [6] | Accuracy: %71.429


IndexError: index 7 is out of bounds for axis 0 with size 7

# **[:+:] Notes**
-   The best single feature is #5, with an accuracy of 0.856

# Works Cited:


-   https://www.analyticsvidhya.com/blog/2020/02/4-types-of-distance-metrics-in-machine-learning/

-   https://www.geeksforgeeks.org/pandas-groupby-count-occurrences-in-column/
-   https://www.codingem.com/python-how-to-get-all-combinations-of-a-list/
-   https://www.geeksforgeeks.org/running-python-script-on-gpu/